In [1]:
# rnn test code

In [2]:
from mnist_read import load_mnist

import tensorflow as tf
import numpy as np

In [3]:
mnist = load_mnist()

 current train_img shape :  (60000, 28, 28, 1)


In [4]:
train_num = 60000
test_num = 10000
batch_size = 100

epoch = 5
data_dims = (28, 28)
cell_dims = [128, 3]
learning_rate = 0.001

In [5]:
X = tf.placeholder(tf.float32, [None, 28, 28])
Y = tf.placeholder(tf.float32, [None, 10])

In [6]:
rnnCell = tf.nn.rnn_cell

In [7]:
cells = [rnnCell.BasicLSTMCell(num_units = n) for n in cell_dims]
multiCell = rnnCell.MultiRNNCell(cells)
outputs, _ = tf.nn.dynamic_rnn(multiCell, X, dtype=tf.float32)

In [8]:
print(outputs)

Tensor("rnn/transpose_1:0", shape=(?, 28, 3), dtype=float32)


In [9]:
outputs = tf.transpose(outputs, [1, 0, 2])[-1]

In [10]:
print(outputs)

Tensor("strided_slice:0", shape=(?, 3), dtype=float32)


In [13]:
fc_outputs = tf.contrib.layers.fully_connected(outputs, 10, activation_fn = None)

In [14]:
print(fc_outputs)

Tensor("fully_connected_1/BiasAdd:0", shape=(?, 10), dtype=float32)


In [15]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits = fc_outputs, labels = Y))

In [16]:
train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [17]:
sess=tf.Session()
sess.run(tf.global_variables_initializer())

In [18]:
total_batch = int(train_num/ batch_size)

In [23]:
for i in range(epoch):
    total_loss = 0
    for j in range(total_batch):
        train_x = mnist['train_img'][0 + (j*batch_size): ((j+1)*batch_size)]
        train_y = mnist['train_label'][0 + (j*batch_size): ((j+1)*batch_size)]
        train_x = np.squeeze(train_x, axis=3)
        
        _, loss_val = sess.run([train_op, loss], feed_dict={X:train_x, Y:train_y})
        total_loss += loss_val
        print(" batch %i/%i loss %f total-loss %f"%(j+1, total_batch, loss_val, total_loss), end="\r")
    print("\n epoch %i | loss %f"%(i+1, total_loss/total_batch))

 batch 600/600 loss 0.597394 total-loss 406.675868
 epoch 0 | loss 0.677793
 batch 600/600 loss 0.509175 total-loss 323.752271
 epoch 1 | loss 0.539587
 batch 600/600 loss 0.471744 total-loss 267.385756
 epoch 2 | loss 0.445643
 batch 600/600 loss 0.368065 total-loss 231.212107
 epoch 3 | loss 0.385354
 batch 600/600 loss 0.353069 total-loss 202.773623
 epoch 4 | loss 0.337956


In [24]:
match = tf.equal(tf.argmax(fc_outputs, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(match, tf.float32))

In [27]:
test_total_batch = int(test_num/batch_size)
sum_acc = 0
for i in range(test_total_batch):
    test_x = mnist['test_img'][0 + (i*batch_size): ((i+1)*batch_size)]
    test_x = np.squeeze(test_x, axis = 3)
    test_y = mnist['test_label'][0 + (i*batch_size): ((i+1)*batch_size)]
    
    pred = sess.run(accuracy, feed_dict={X:test_x, Y:test_y})
    sum_acc += pred
    print(" batch %i/%i acc %f"%(i+1, test_total_batch, pred), end = "\r")
print("\n accuracy = %f"%(sum_acc/test_total_batch))

 batch 100/100 acc 0.930000
 accuracy = 0.884700
